In [4]:
from parse import *

In [5]:
def create_log_list(log_file):
    logs = open(log_file)
    lines  = logs.readlines()
    log_list = []
    log = ''
    for line in lines:
        if line[:11] == "2014/Oct/24":
            log_list.append(log)
            log = line
        else:
            log += line

    log_list = log_list[1:]
    return log_list

log_list = create_log_list('WCG100140020.txt')

In [6]:
def portevent_filter(log_list):
    portevent_log_list = []
    file = open('porteventlogs.txt', 'w')
    for log in log_list:
        if "PORTEVENT" in log:
            log_line = log.replace('\n', '')
            portevent_log_list.append(log_line)
            file.write(log_line)
    return portevent_log_list
def sending_message_filter(log_list):
    sending_log_list = []
    for log in log_list:
        if "Sent on" in log:
            sending_log_list.append(log)
    return sending_log_list

portevent_log_list  = portevent_filter(log_list)
sending_log_list = sending_message_filter(log_list)


In [7]:
from parse import compile
from parse import with_pattern
import datetime as dt

@with_pattern(r"\d\d\d\d/\d\d/\d\d \d\d:\d\d:\d\d.\d\d\d\d\d\d")
def date_format(date_time_str):
    return dt.datetime.strptime(date_time_str, '%Y/%b/%d %H:%M:%S.%f')

port_opening = compile("{date:27} {user_name} {eventtype} {unknown_part} Port {portname} was started.")
port_stopping = compile("{date:27} {user_name} {eventtype} {unknown_part} Port {portname} was stopped.")
port_connexion_waiting = compile("{date:27} {message_receiver} {eventtype} {unknown_part} Port {message_receiver_portname} is waiting for connection from {message_sender_portname} on {communication_type} pathname {pathname}.")
port_connexion_establishment = compile("{date:27} {message_sender} {eventtype} {unknown_part} Port {message_sender_portname} has established the connection with {message_receiver_portname} using transport type {communication_type}.")
port_connexion_acceptation = compile("{date:27} {message_receiver} {eventtype} {unknown_part} Port {message_receiver_portname} has accepted the connection from {message_sender_portname}.")
port_mapping = compile("{date:27} {message_sender} {eventtype} {unknown_part} Port {message_sender_portname} was mapped to {message_receiver_portname}.")
message_sending = compile("{date:27} {message_sender} {eventtype} {unknown_part} Sent on {message_sender_portname} to {message_receiver} {message_type} : {message_content:0}")
message_enqueuing = compile("{date:27} {message_receiver} {eventtype} {unknown_part} Message enqueued on {message_receiver_portname} from {message_sender} {message_type} : {message_content:0} id {message_id_number}")
message_reception = compile("{date:27} {message_receiver} {eventtype} {unknown_part} Receive operation on port {message_receiver_portname} succeeded, message from {message_sender}: {message_type} : {message_content:0} id {message_id_number}")
message_extraction = compile("{date:27} {message_receiver} {eventtype} {unknown_part} Message with id {message_id_number} was extracted from the queue of {message_receiver_portname}.")

print(port_opening.parse(portevent_log_list[0]))
print(port_stopping.parse("2014/Oct/24 18:34:38.966352 mtc PORTEVENT WCG10014_0020.ttcn:129(testcase:WCG100140020) Port tcpClientPort was stopped."))
print(port_connexion_waiting.parse("2014/Oct/24 18:34:28.683427 mtc PORTEVENT SipClientLayerComponent.ttcn:253(function:receiveInternalMessage) Port sipInternalPort[0] is waiting for connection from 849:sipInternalPort on UNIX pathname /tmp/ttcn3-portconn-2256f413."))
print(port_connexion_establishment.parse("2014/Oct/24 18:34:28.683552 849 PORTEVENT sipClientInterface.ttcn:146(function:sipInitialize) Port sipInternalPort has established the connection with mtc:sipInternalPort[0] using transport type UNIX."))
print(port_connexion_acceptation.parse("2014/Oct/24 18:34:28.683618 mtc PORTEVENT SipClientLayerComponent.ttcn:253(function:receiveInternalMessage) Port sipInternalPort[0] has accepted the connection from 849:sipInternalPort."))
print(port_mapping.parse("2014/Oct/24 18:34:28.683908 849 PORTEVENT sipClientInterface.ttcn:149(function:sipInitialize) Port sipClientPort was mapped to system:sipClientPort[0]."))


<Result () {'date': '2014/Oct/24 18:34:28.666455', 'user_name': 'mtc', 'eventtype': 'PORTEVENT', 'unknown_part': 'WCG10014_0020.ttcn:53(testcase:WCG100140020)', 'portname': 'dnsInternalPort[0]'}>
<Result () {'date': '2014/Oct/24 18:34:38.966352', 'user_name': 'mtc', 'eventtype': 'PORTEVENT', 'unknown_part': 'WCG10014_0020.ttcn:129(testcase:WCG100140020)', 'portname': 'tcpClientPort'}>
<Result () {'date': '2014/Oct/24 18:34:28.683427', 'message_receiver': 'mtc', 'eventtype': 'PORTEVENT', 'unknown_part': 'SipClientLayerComponent.ttcn:253(function:receiveInternalMessage)', 'message_receiver_portname': 'sipInternalPort[0]', 'message_sender_portname': '849:sipInternalPort', 'communication_type': 'UNIX', 'pathname': '/tmp/ttcn3-portconn-2256f413'}>
<Result () {'date': '2014/Oct/24 18:34:28.683552', 'message_sender': '849', 'eventtype': 'PORTEVENT', 'unknown_part': 'sipClientInterface.ttcn:146(function:sipInitialize)', 'message_sender_portname': 'sipInternalPort', 'message_receiver_portname':

In [8]:
import pandas as pd

parsed_sending_log_dico = {'date': [], 'message_sender': [], 'eventtype': [], 'unknown_part': [], 'message_sender_portname': [], 
                           'message_receiver': [], 'message_type': [], 'message_content': []}
for sending_log in sending_log_list:
    parsed_log = message_sending.parse(sending_log).named
    for k in parsed_log.keys():
        parsed_sending_log_dico[k].append(parsed_log[k])
df = pd.DataFrame(parsed_sending_log_dico)
df.head(5)

,date,message_sender,eventtype,unknown_part,message_sender_portname,message_receiver,message_type,message_content
0,2014/Oct/24 18:34:28.684126,849,PORTEVENT,sipClientInterface.ttcn:152(function:sipInitia...,sipInternalPort,mtc,@variables.internalPortMessageWithAspsSip,{\n internalMessage := {\n descripti...
1,2014/Oct/24 18:34:28.695096,mtc,PORTEVENT,SipClientLayerComponent.ttcn:202(function:send...,sipInternalPort[0],849,@variables.internalPortMessageWithAspsSip,{\n internalMessage := {\n descripti...
2,2014/Oct/24 18:34:28.696199,849,PORTEVENT,sipClientInterface.ttcn:388(function:processCo...,sipInternalPort,mtc,@variables.internalPortMessageWithAspsSip,{\n internalMessage := {\n descripti...
3,2014/Oct/24 18:34:28.700272,850,PORTEVENT,msrpInterface.ttcn:161(function:msrpInitialize),msrpInternalPort,mtc,@variables.internalPortMessageWithMsrpMessages,{\n internalMessage := {\n descripti...
4,2014/Oct/24 18:34:28.711152,mtc,PORTEVENT,MsrpLayerComponent.ttcn:168(function:sendInter...,msrpInternalPort[0],850,@variables.internalPortMessageWithMsrpMessages,{\n internalMessage := {\n descripti...


In [9]:
for m in df.message_content:
    print(m)

{
    internalMessage := {
        description := "COMPONENT_UP_AND_RUNNING",
        parameters := { }
    },
    aspsSip := {
        {
            dummy := ""
        }
    }
}

{
    internalMessage := {
        description := "COMMAND_START_LISTENER",
        parameters := {
            {
                parameterName := "STEP_NUMBER",
                parameterValue := "0"
            },
            {
                parameterName := "SUBSCRIBER",
                parameterValue := "11"
            }
        }
    },
    aspsSip := {
        {
            dummy := ""
        }
    }
}

{
    internalMessage := {
        description := "COMPONENT_DONE_WITH_SUCCESS",
        parameters := {
            {
                parameterName := "COMMAND_EXECUTED",
                parameterValue := "COMMAND_START_LISTENER"
            }
        }
    },
    aspsSip := {
        {
            dummy := ""
        }
    }
}

{
    internalMessage := {
        description := "COMPONENT_UP_AND_RUN

In [10]:
def edit_dict_string(string):
    lines = string.split('\n')
    edited_message = ''
    for line in lines:
        decompose_line = line.split(' ')
        edited_line = ''
        for i in range(len(decompose_line)):
            if decompose_line[i] != '' and decompose_line[i] != ':=' and decompose_line[i] != '{' and decompose_line[i] != '}' and decompose_line[i] != '},':
                if decompose_line[i] != '"[':
                    if decompose_line[i][-1] == ',':
                        decompose_line[i] = '"' + decompose_line[i].replace('"', '').replace(',','') + '",'
                    else :
                        decompose_line[i] = '"' + decompose_line[i].replace('"', '') + '"'
                else:
                    break
        for e in decompose_line:
            if e == '':
                edited_line += ' '
            if e == ':=':
                edited_line += ' ' + e + ' '
            else:
                edited_line += e
        edited_message += edited_line + '\n'
    return edited_message


s = df.message_content.iloc[10]
        
print(edit_dict_string(s))

{
    "internalMessage" := {
        "description" := "COMMAND_HTTP_CLIENT_SEND_GET_ANY_MESSAGE",
        "parameters" := {
            {
                "parameterName" := "PARAM_PORT_OVERRIDE",
                "parameterValue" := "8888"
            }
        }
    },
    "aspHttp" := {
        {
            "httpMessage" := {
                "method" := "POST",
                "url" := "/jolokia/",
                "body" := "[{\"type\":\"read\",\"mbean\":\"com.ericsson.caf.pm:Application=Wcg,interface=WcgChatCounterGroup,type=CC,successfulTerminating=successfulTerminating-1\",\"attribute\":\"Value\",\"config\":{\"mimeType\":\"application/json\",\"includeStackTrace\":\"false\"}},{\"type\":\"read\",\"mbean\":\"com.ericsson.caf.pm:Application=Wcg,interface=WcgChatCounterGroup,type=CC,successfulOriginating=successfulOriginating-1\",\"attribute\":\"Value\",\"config\":{\"mimeType\":\"application/json\",\"includeStackTrace\":\"false\"}},{\"type\":\"read\",\"mbean\":\"com.ericsson.caf.pm:App

In [15]:
import json

def string_to_dict(string_dico):
    string_dico = string_dico.split(' ')
    open = 0
    ind = []
    for i in range(len(string_dico)):
        if '{' in string_dico[i]:
            open += 1
            f = list(filter(('').__ne__, string_dico[i:]))
            if len(f) > 0:
                if '{' in f[1]:
                    string_dico[i] = string_dico[i].replace('{', '[',-1)
                    ind.appen(open)
        if '}' in string_dico[i]:
            if open == ind.pop(0):
                string_dico[i] = string_dico[i].replace('}',']',1)
            open -= 1

    for i in range(len(string_dico)):
        if string_dico[i] == '':
            string_dico[i] = ' '

    string = """"""
    for i in range(len(string_dico)):
        string += string_dico[i]
    string = string.replace('=', '')
    print(string)
    string_dico_json = json.loads(string)
    return string_dico_json

In [12]:
def test_message(ind):
    string  = df.message_content.iloc[ind]
    print(string)
    edited_string = edit_dict_string(string)
    print(edited_string)
    dico = string_to_dict(edited_string)
    print(dico)

In [13]:
def message_content_to_dict(message_list):
    parsed_message_list = []
    for i,m in enumerate(message_list):
        try:
            parsed_message_list.append(string_to_dict(edit_dict_string(m)))
        except:
            print(i)
    return parsed_message_list

parsed_message_list = message_content_to_dict(df.message_content)

13
17
20
22
24
25
26
29
30
31
33
36
38
40
41
42
45
46
47
49
50
52
53
55
56
57
59
60
63
65
71
73
74
75
78
79
80
87
88
90
91
93
95
96
99
100
101
103
104
107
108
109
112
114
115
116
118
119
122
126
128
129
130
132
133
136
140
144
145
147
148
150
151
152
154
155
158
162
164
165
166
168
169
172
176
177
179
180
182
183
184
186
187
190
194
196
197
198
200
201
204
208
212
220
221
222
223
226
227
228
230
232
233
234
242
243
245
246
247
252
256
264


In [16]:
test_message(13)

{
    internalMessage := {
        description := "COMPONENT_DONE_WITH_SUCCESS",
        parameters := {
            {
                parameterName := "COMMAND_EXECUTED",
                parameterValue := "COMMAND_HTTP_CLIENT_SEND_GET_ANY_MESSAGE"
            }
        }
    },
    aspHttp := {
        {
            httpMessage := {
                method := "",
                url := "",
                body := "[{\"timestamp\":1414190068,\"status\":200,\"request\":{\"mbean\":\"com.ericsson.caf.pm:Application=Wcg,interface=WcgChatCounterGroup,successfulTerminating=successfulTerminating-1,type=CC\",\"attribute\":\"Value\",\"type\":\"read\"},\"value\":0},{\"timestamp\":1414190068,\"status\":200,\"request\":{\"mbean\":\"com.ericsson.caf.pm:Application=Wcg,interface=WcgChatCounterGroup,successfulOriginating=successfulOriginating-1,type=CC\",\"attribute\":\"Value\",\"type\":\"read\"},\"value\":0},{\"timestamp\":1414190068,\"status\":200,\"request\":{\"mbean\":\"com.ericsson.caf.pm:Applica

JSONDecodeError: Invalid \escape: line 16 column 36 (char 405)

In [12]:
print(df.message_content.iloc[12])
print(getdico(df.message_content.iloc[12]))

{
    request := {
        client_id := 9,
        method := "POST",
        uri := "/jolokia/",
        version_major := 1,
        version_minor := 1,
        header := {
            {
                header_name := "Host",
                header_value := "142.133.142.3"
            },
            {
                header_name := "Content-Type",
                header_value := "application/json"
            },
            {
                header_name := "Content-Length",
                header_value := "1971"
            }
        },
        body := "[ { \"type\": \"read\", \"mbean\": \"com.ericsson.caf.pm:Application=Wcg,interface=WcgChatCounterGroup,type=CC,successfulTerminating=successfulTerminating-1\", \"attribute\": \"Value\", \"config\": { \"mimeType\": \"application/json\", \"includeStackTrace\": \"false\"} } , { \"type\": \"read\", \"mbean\": \"com.ericsson.caf.pm:Application=Wcg,interface=WcgChatCounterGroup,type=CC,successfulOriginating=successfulOriginating-1\", \"attrib